In [ ]:
### 스타벅스의 매장 입지에 대한 2가지 가설
#  1. 거주 인구가 많은 지역에 스타벅스 매장이 많이 입지해 있을 것이다. 
#  2. 직장인이 많은 지역에 스타벅스 매장이 많이 입지해 있을 것이다. 

In [ ]:
### 데이터 수집(서울시 내)
# -1. 스타벅스들의 위치
# -2. 인구통계 데이터 수집

In [1]:
## 스타벅스 매장 목록 데이터 생성
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

### driver
driver = webdriver.Chrome("C:/webdriver/chromedriver.exe")
url = 'https://www.starbucks.co.kr/store/store_map.do?disp=locale'
driver.get(url)

C:\Users\vmuser\AppData\Local\Temp\ipykernel_5264\1017888748.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("C:/webdriver/chromedriver.exe")


In [2]:
seoul_btn = "#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a"
driver.find_element_by_css_selector(seoul_btn).click()

C:\Users\vmuser\AppData\Local\Temp\ipykernel_5264\3850842969.py:2: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  driver.find_element_by_css_selector(seoul_btn).click()


In [3]:
all_btn = "#mCSB_2_container > ul > li:nth-child(1) > a"
driver.find_element_by_css_selector(all_btn).click()

C:\Users\vmuser\AppData\Local\Temp\ipykernel_5264\3624987384.py:2: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  driver.find_element_by_css_selector(all_btn).click()


In [4]:
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

In [5]:
starbucks_soup_list = soup.select('li.quickResultLstCon')
print(len(starbucks_soup_list))
# starbucks_soup_list

578


In [35]:
### 매장명, 위도, 경도, 매장타입, 주소, 전화번호 
# print(starbucks_soup_list[0])
starbucks_store = starbucks_soup_list[0]
name = starbucks_store.select('strong')[0].text.strip()
lat = starbucks_store['data-lat'].strip()
long = starbucks_store['data-long'].strip()
store_type = starbucks_store.select('i')[0]['class'][0][4:]
address = str(starbucks_store.select('p.result_details')[0]).split('<br/>')[0].split('>')[1]
tel = str(starbucks_store.select('p.result_details')[0]).split('<br/>')[1].split('<')[0]

print(name)
print(lat)
print(long)
print(store_type)
print(address)
print(tel)


역삼아레나빌딩
37.501087
127.043069
general
서울특별시 강남구 언주로 425 (역삼동)
1522-3232


In [38]:
## 스타벅스 매장 목록 데이터 생성
starbucks_list = []
for item in starbucks_soup_list:
    name = item.select('strong')[0].text.strip();
    lat = item['data-lat'].strip()
    long = item['data-long'].strip()
    store_type = item.select('i')[0]['class'][0][4:]
    address = str(item.select('p.result_details')[0]).split('<br/>')[0].split('>')[1]
    tel = str(item.select('p.result_details')[0]).split('<br/>')[1].split('<')[0]
    
    starbucks_list.append([name, lat, long, store_type, address, tel])


['역삼아레나빌딩',
 '37.501087',
 '127.043069',
 'general',
 '서울특별시 강남구 언주로 425 (역삼동)',
 '1522-3232']

In [41]:
### DataFrame으로 만들기
columns = ['매장명','위도','경도','매장타입','주소','전화번호']
seoul_starbucks_df = pd.DataFrame(starbucks_list,columns=columns)
seoul_starbucks_df.info()
seoul_starbucks_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 578 entries, 0 to 577
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   매장명     578 non-null    object
 1   위도      578 non-null    object
 2   경도      578 non-null    object
 3   매장타입    578 non-null    object
 4   주소      578 non-null    object
 5   전화번호    578 non-null    object
dtypes: object(6)
memory usage: 27.2+ KB


,매장명,위도,경도,매장타입,주소,전화번호
0,역삼아레나빌딩,37.501087,127.043069,general,서울특별시 강남구 언주로 425 (역삼동),1522-3232
1,논현역사거리,37.510178,127.022223,general,서울특별시 강남구 강남대로 538 (논현동),1522-3232
2,신사역성일빌딩,37.514132,127.020563,general,서울특별시 강남구 강남대로 584 (논현동),1522-3232
3,국기원사거리,37.499517,127.031495,general,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232
4,대치재경빌딩R,37.494668,127.062583,reserve,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232


In [42]:
seoul_starbucks_df.to_excel("data_starbucks/seoul_starbucks_list.xlsx",index=False)

In [49]:
#### 서울 열린 데이터광장 공공데이터 수집 및 처리
## 주민등록인구 통계 데이터 
sgg_pop_df = pd.read_csv("data_starbucks/report.txt", sep="\t",header=2)
columns = {
    '기간':'GIGAN',
    '자치구':'JACHIGU',
    '계': 'GYE_1',
    '계.1': 'GYE_2',
    '계.2': 'GYE_3',
    '남자': 'NAMJA_1',
    '남자.1': 'NAMJA_2',
    '남자.2': 'NAMJA_3',
    '여자': 'YEOJA_1',
    '여자.1': 'YEOJA_2',
    '여자.2': 'YEOJA_3',
    '세대': 'SEDAE',
    '세대당인구': 'SEDAEDANGINGU',
    '65세이상고령자': 'N 65SEISANGGORYEONGJA'
}
# print(sgg_pop_df)
sgg_pop_df.rename(columns=columns, inplace = True)
sgg_pop_df.info()
sgg_pop_df



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   GIGAN                  26 non-null     object 
 1   JACHIGU                26 non-null     object 
 2   SEDAE                  26 non-null     object 
 3   GYE_1                  26 non-null     object 
 4   NAMJA_1                26 non-null     object 
 5   YEOJA_1                26 non-null     object 
 6   GYE_2                  26 non-null     object 
 7   NAMJA_2                26 non-null     object 
 8   YEOJA_2                26 non-null     object 
 9   GYE_3                  26 non-null     object 
 10  NAMJA_3                26 non-null     object 
 11  YEOJA_3                26 non-null     object 
 12  SEDAEDANGINGU          26 non-null     float64
 13  N 65SEISANGGORYEONGJA  26 non-null     object 
dtypes: float64(1), object(13)
memory usage: 3.0+ KB


,GIGAN,JACHIGU,SEDAE,GYE_1,NAMJA_1,YEOJA_1,GYE_2,NAMJA_2,YEOJA_2,GYE_3,NAMJA_3,YEOJA_3,SEDAEDANGINGU,N 65SEISANGGORYEONGJA
0,2022.1/4,합계,"4,451,444","9,732,617","4,716,704","5,015,913","9,506,778","4,613,910","4,892,868","225,839","102,794","123,045",2.14,"1,628,980"
1,2022.1/4,종로구,"73,766","153,684","74,018","79,666","144,275","69,911","74,364","9,409","4,107","5,302",1.96,"28,064"
2,2022.1/4,중구,"63,767","131,450","63,869","67,581","122,173","59,448","62,725","9,277","4,421","4,856",1.92,"24,742"
3,2022.1/4,용산구,"111,207","236,518","114,571","121,947","222,169","106,735","115,434","14,349","7,836","6,513",2.00,"39,461"
4,2022.1/4,성동구,"134,533","291,609","141,662","149,947","285,073","138,847","146,226","6,536","2,815","3,721",2.12,"46,896"
5,2022.1/4,광진구,"169,483","353,110","169,525","183,585","340,707","164,346","176,361","12,403","5,179","7,224",2.01,"52,604"
6,2022.1/4,동대문구,"169,695","352,592","172,318","180,274","337,982","166,823","171,159","14,610","5,495","9,115",1.99,"62,963"
7,2022.1/4,중랑구,"187,693","393,600","193,623","199,977","389,043","191,835","197,208","4,557","1,788","2,769",2.07,"72,851"
8,2022.1/4,성북구,"197,296","442,680","212,277","230,403","433,150","208,691","224,459","9,530","3,586","5,944",2.20,"75,719"
9,2022.1/4,강북구,"144,725","300,824","145,934","154,890","297,384","144,643","152,741","3,440","1,291","2,149",2.05,"64,961"


In [60]:
## 필요없는 데이터 제거, 합계(자치구) 제거
condition = sgg_pop_df['JACHIGU'] != '합계'
sgg_pop_df_selected = sgg_pop_df[condition]
sgg_pop_df_selected

## 분석에 필요한 컬럼
columns = ['JACHIGU','GYE_1']
sgg_pop_df_final = sgg_pop_df_selected[columns]
sgg_pop_df_final.info()
sgg_pop_df_final.head()

## 데이터프레임 컬럼명 변경
sgg_pop_df_final.columns = ['시군구명','주민등록인구']
sgg_pop_df_final.info()
sgg_pop_df_final.head()

sgg_pop_df_final.to_excel("data_starbucks/sgg_pop.xlsx",index=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 1 to 25
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   JACHIGU  25 non-null     object
 1   GYE_1    25 non-null     object
dtypes: object(2)
memory usage: 600.0+ bytes
<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 1 to 25
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   시군구명    25 non-null     object
 1   주민등록인구  25 non-null     object
dtypes: object(2)
memory usage: 600.0+ bytes


In [59]:
## 서울시 동별 사업체현황 통계
sgg_biz_df = pd.read_csv("data_starbucks/report2.txt",sep="\t",header=2)
columns = {
    '기간':'GIGAN',
    '자치구':'JACHIGU',
    '동':'DONG',
    '사업체수':'SAEOPCHESU_1',
    '여성대표자':'YEOSEONGDAEPYOJA',
    '계':'GYE',
    '남':'NAM',
    '여':'YEO'
}
sgg_biz_df.rename(columns=columns,inplace=True)
sgg_biz_df.head(10)

## 시군구별 사업체 현황 데이터 추출(동별X)
condition = sgg_biz_df['DONG'] == "소계"
sgg_biz_df_selected = sgg_biz_df[condition]
sgg_biz_df_selected.head()

## 필요없는 컬럼 제거
columns = ['JACHIGU','GYE','SAEOPCHESU_1']
sgg_biz_df_final = sgg_biz_df_selected[columns]
sgg_biz_df_final.columns = ['시군구명','종사자수','사업체수']
sgg_biz_df_final.head()

## 인덱스 초기화
sgg_biz_df_final = sgg_biz_df_final.reset_index(drop=True)
sgg_biz_df_final.head()

,시군구명,종사자수,사업체수
0,종로구,"260,446","39,679"
1,중구,"392,568","60,127"
2,용산구,"135,240","20,254"
3,성동구,"178,243","28,343"
4,광진구,"126,305","24,445"


In [61]:
## 엑셀로 저장
sgg_biz_df_final.to_excel("data_starbucks/sgg_biz.xlsx",index=False)

In [ ]:
#### 1. 서울시 인구 조사(시군구별) - data_starbucks/sgg_pop.xlsx
#### 2. 서울시 사업체데이터를 수집 - data_starbucks/sgg_biz.xlsx
#### 3. 스타벅스 매장 목록 - data_starbucks/seoul_starbucks_list.xlsx
